In [7]:
import numpy as np
import tensorflow as tf
tf.random.set_seed(123)
from BGL.bglog import BGLog, get_embedding_layer
from pretraining import LogLineEncoder, LogSeqEncoder, LogClassifier
from boundary_loss import euclidean_metric, BoundaryLoss

In [8]:
bglog = BGLog(save_padded_num_sequences=False, load_from_pkl=True)

In [9]:
train_test = bglog.get_tensor_train_test(ablation=1000)
train_data, test_data = train_test

padded_num_seq_df loaded from data\bgl_padded_num_seq_df.pkl
trained tokenizer, tk, loaded from data\bgltk.pkl
train_0:, 800
test_0:, 200
train_1:, 800
test_1:, 200
train_2:, 800
test_2:, 200
train_3:, 800
test_3:, 102
4 class does not have 800 records, it has only 628 records
test_4:, 0
5 class does not have 800 records, it has only 165 records
5 class does not have 200 records, it has only 165 records
6 class does not have 800 records, it has only 75 records
6 class does not have 200 records, it has only 75 records
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]]
<BatchDataset shapes: ((32, 32, 64), (32, 4)), types: (tf.int32, tf.float32)>
<BatchDataset shapes: ((32, 32, 64), (32, 4)), types: (tf.int32, tf.float32)>


In [10]:
line_encoder =   LogLineEncoder(bglog, chars_in_line=64)
# the model doesn't have a state unless it is called at least once
# in order to initialize the model we need a sample data 
sample_train_data = next(iter(train_data))
sample_x_train = sample_train_data[0]
print('sample_x_train.shape:', sample_x_train.shape)
# now we will initialize the model with the sample data
loglineEmbedding = line_encoder(sample_x_train)
print('loglineEmbedding.shape:', loglineEmbedding.shape)
# Now the model have a state and can be inspected        
# line_encoder.summary()

vocab_size: 50
sample_x_train.shape: (32, 32, 64)
loglineEmbedding.shape: (32, 32, 64)


In [11]:
logSeqencer =   LogSeqEncoder(line_in_seq=32)
# the model doesn't have a state unless it is called at least once
logSeqEmbedding = logSeqencer(loglineEmbedding)
print('logSeqEmbedding.shape:', logSeqEmbedding.shape)
# Now the model have a state and can be inspected        
# logSeqencer.summary()

logSeqEmbedding.shape: (32, 16)


In [12]:
log_classifier = LogClassifier(line_encoder=line_encoder, seq_encoder=logSeqencer, num_classes=4)
# log_classifier(sample_x_train)  

In [13]:
# log_classifier.summary()

In [14]:
log_classifier.compile(optimizer='adam', 
                  loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
hist = log_classifier.fit(train_data, validation_data=test_data, epochs=1) 

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [9]:
# log_classifier(sample_x_train)   

In [15]:
class OpenSet:
    ''' 
    self.num_labels = number of classes
    self.embedding_size = number of neurons in the logits layers of the pretrained model'''
    def __init__(self, num_labels, pretrained_model, embedding_size,
                lr_boundary):
#         super().__init__():
        self.model = pretrained_model        
        self.centroids = None
        self.num_labels = num_labels
        self.embedding_size = embedding_size
        self.delta = None
        self.lr_boundary = lr_boundary
        self.delta_points = []
        
    
    def train(self, data_train, epochs=1):
        criterion_boundary = BoundaryLoss(num_labels=self.num_labels)
        # delta is getting calculated inside the  BoundaryLoss class as well
        # however that calculated delta is used for calculating the loss 
        # that delta is not updating the criterion_boundary.delta which is 
        # a randomly initialized parameter. 
        # Hence the following softplus is on randomly initialized trainable parameters
        # and not softplus on softplus
        self.delta = tf.nn.softplus(criterion_boundary.delta)
        self.centroids = self.centroids_cal(data_train)        
        optimizer = tf.keras.optimizers.Adam(learning_rate=self.lr_boundary) # does it take criterion_boundary.parameters() ??
        wait = 0
        best_delta, best_centroids = None, None
        
        for epoch in range(epochs):
            tr_loss = 0
            nb_tr_examples, nb_tr_steps = 0, 0
            for batch in data_train:
                logseq_batch, label_batch = batch
                ## (32, 32, 64), (32, 4)
                batch_loss, t_loss = self.train_step(criterion_boundary, 
                                                     logseq_batch, label_batch, optimizer)
                tr_loss += t_loss
                nb_tr_steps += 1
                
            self.delta_points.append(self.delta)
            loss = tr_loss / nb_tr_steps
            print('train_loss:', loss)  
                    
#     @tf.function                
    def train_step(self, criterion_boundary, logseq_batch, label_batch, optimizer):
#         print('within train_step')
        tr_loss = 0
        with tf.GradientTape() as tape:                
            features_batch = self.model(logseq_batch, extract_feature=True)
            loss, self.delta = criterion_boundary(features_batch, 
                                                  self.centroids, 
                                                  label_batch)
        tr_loss += loss
        gradients = tape.gradient(loss, [self.delta])

        optimizer.apply_gradients(zip(gradients, [self.delta]))
        return loss, tr_loss                    
        
        
        
    def centroids_cal(self, data):
        centroids = tf.zeros((self.num_labels, self.embedding_size))
        total_labels = tf.zeros(self.num_labels)
        for batch in data:
            logseq_batch, label_batch = batch
            ## (32, 32, 64), (32, 4)
            features = self.model(logseq_batch, extract_feature=True)
            ## (32, 16) features - 32 sequence of line each haaving 64 characrers
            ## produces a feaure vector of dimension 16. 
            for i in range(len(label_batch)): # (32, 4) --> here length is 32
                label = label_batch[i] # label looks like [0 0 0 1]
                numeric_label = np.argmax(label) # index position of the label = 3 , so it is actually class =3
                ##total_labels = [0 0 0 0] each col representing a class 
                ## count the number for each class
                total_labels_lst = tf.unstack(total_labels)
                total_labels_lst[numeric_label] += 1 
                total_labels = tf.stack(total_labels_lst)
                centroids_lst = tf.unstack(centroids)
                centroids_lst[numeric_label] += features[i]
                centroids = tf.stack(centroids_lst)
                # each row index in the centroid array is a class
                # we add first identify the feature belonging to which class by the numeric_label
                # Then add all the features belonging to the class in the corresponding row of the centroid arr
        ### shape of centroids is (4, 16) whereas shape of total_labels is (1, 4)
        ### reshape the total_labels as 4,1 ==> [[0], [0], [0], [0]]==> 4 rows 
        ## so that we can divide the centroids array by the total_labels
        total_label_reshaped = tf.reshape(total_labels, (self.num_labels, 1))
        centroids /= total_label_reshaped
        return centroids  
        
        def openpredict(self, features):
            logits = euclidean_metric(features, self.centroids)
            ####original line in pytorch ###probs, preds = F.softmax(logits.detach(), dim = 1).max(dim = 1)
            smax = tf.nn.softmax(logits, )
            preds = tf.math.argmax(smax, axis=1)
            probs = tf.reduce_max(smax, 1)            
            #######euc_dis = torch.norm(features - self.centroids[preds], 2, 1).view(-1)
            euc_dis = tf.norm(features - self.centroids[preds], ord='euclidean', axis=1) 
            
            
            #preds[euc_dis >= self.delta[preds]] = data.unseen_token_id

            return preds
        

In [16]:
# loss, self.delta = criterion_boundary(features_batch,
# logits =  euclidean_metric(features, centroids)
# NotImplementedError: Cannot convert a symbolic Tensor (log_classifier/log_seq_encoder/dense/Relu:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported 
# it looks like the numpy arrays to be converted to tensor

In [17]:
oset = OpenSet(4, log_classifier, 16, 0.05)

In [18]:
# optimizer.apply_gradients(zip(gradients, self.delta))
# TypeError: 'IndexedSlices' object is not iterable

In [19]:
t_batch = next(iter(train_data))
t_batch_x, t_batch_y = t_batch
t_batch_x.shape
centroids = oset.centroids_cal(train_data)
features_batch = log_classifier(t_batch_x, extract_feature=True)


In [20]:
criterion_boundary = BoundaryLoss(num_labels=4)
loss, delta = criterion_boundary(features_batch, centroids, t_batch_y)

In [21]:
loss

<tf.Tensor: shape=(32,), dtype=float32, numpy=
array([1.6548097, 1.6548097, 1.6548097, 1.6548097, 1.6931441, 1.6931441,
       1.6548097, 1.6362237, 1.6362237, 1.6548097, 1.6548097, 1.6931441,
       1.6754997, 1.6754997, 1.6362237, 1.6931441, 1.6362237, 1.6362237,
       1.6931441, 1.6548097, 1.6754997, 1.6754997, 1.6931441, 1.6754997,
       1.6548097, 1.6548097, 1.6548097, 1.6548097, 1.6754997, 1.6548097,
       1.6754997, 1.6362237], dtype=float32)>

In [22]:
delta

<tf.Variable 'boundary_loss/Variable:0' shape=(4, 1) dtype=float32, numpy=
array([[0.6644433],
       [0.6851332],
       [0.7037192],
       [0.6467987]], dtype=float32)>

In [23]:
tr_loss=0
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
with tf.GradientTape() as tape:                
    features_batch = log_classifier(t_batch_x, extract_feature=True)
    loss, delta = criterion_boundary(features_batch, centroids, t_batch_y)
tr_loss += loss
gradients = tape.gradient(loss, [delta])


In [24]:
delta = tf.Variable(delta)
delta

<tf.Variable 'Variable:0' shape=(4, 1) dtype=float32, numpy=
array([[0.6644433],
       [0.6851332],
       [0.7037192],
       [0.6467987]], dtype=float32)>

In [25]:
gradients

In [26]:
# optimizer.minimize(loss, delta)

In [27]:
# optimizer.apply_gradients(zip(gradients, delta))
#### AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute '_in_graph_mode'

In [28]:
# @tf.function
def check_gradient(t_batch_x, t_batch_y, centroids):
    tr_loss=0
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
    with tf.GradientTape() as tape:                
        features_batch = log_classifier(t_batch_x, extract_feature=True)
        loss, delta = criterion_boundary(features_batch, centroids, t_batch_y)
    tr_loss += loss
    gradients = tape.gradient(loss, [delta])    
    optimizer.apply_gradients(zip(gradients, [delta]))
#     optimizer.minimize(gradients, var_list=[delta])
    return loss, tr_loss

In [29]:
check_gradient(t_batch_x, t_batch_y, centroids)
###AttributeError: 'Tensor' object has no attribute '_in_graph_mode'

(<tf.Tensor: shape=(32,), dtype=float32, numpy=
 array([1.6548097, 1.6548097, 1.6548097, 1.6548097, 1.6931441, 1.6931441,
        1.6548097, 1.6362237, 1.6362237, 1.6548097, 1.6548097, 1.6931441,
        1.6754997, 1.6754997, 1.6362237, 1.6931441, 1.6362237, 1.6362237,
        1.6931441, 1.6548097, 1.6754997, 1.6754997, 1.6931441, 1.6754997,
        1.6548097, 1.6548097, 1.6548097, 1.6548097, 1.6754997, 1.6548097,
        1.6754997, 1.6362237], dtype=float32)>,
 <tf.Tensor: shape=(32,), dtype=float32, numpy=
 array([1.6548097, 1.6548097, 1.6548097, 1.6548097, 1.6931441, 1.6931441,
        1.6548097, 1.6362237, 1.6362237, 1.6548097, 1.6548097, 1.6931441,
        1.6754997, 1.6754997, 1.6362237, 1.6931441, 1.6362237, 1.6362237,
        1.6931441, 1.6548097, 1.6754997, 1.6754997, 1.6931441, 1.6754997,
        1.6548097, 1.6548097, 1.6548097, 1.6548097, 1.6754997, 1.6548097,
        1.6754997, 1.6362237], dtype=float32)>)

In [30]:
oset.train(train_data)

train_loss: tf.Tensor(
[1.7890495 1.7961293 1.7964641 1.8004185 1.7929924 1.7974381 1.795191
 1.7931007 1.7890009 1.7863306 1.7911556 1.7913154 1.7929295 1.7916319
 1.793261  1.7944844 1.7919563 1.7936623 1.7925035 1.79865   1.7968493
 1.8006352 1.7996067 1.7871317 1.7890861 1.7963542 1.7880127 1.7968636
 1.7952195 1.7908095 1.7912313 1.7910193], shape=(32,), dtype=float32)


In [31]:
oset.delta_points

[<tf.Variable 'boundary_loss_1/Variable:0' shape=(4, 1) dtype=float32, numpy=
 array([[0.71801656],
        [0.7913795 ],
        [0.697228  ],
        [0.76913875]], dtype=float32)>]

In [35]:
logits = euclidean_metric(features_batch, oset.centroids)
print(logits)

tf.Tensor(
[[ -67.01952     -3.525845  -109.583534  -160.43402  ]
 [ -79.82242     -2.1222196  -93.719376  -131.56161  ]
 [-184.88982    -24.641348  -132.53224   -195.33246  ]
 [ -76.3061      -2.9903688 -107.254425  -152.44286  ]
 [-119.22389   -140.64474    -50.575027   -47.487274 ]
 [-222.22762   -185.70552   -101.49874     -6.754195 ]
 [ -70.32373     -1.5167881  -95.82221   -140.16188  ]
 [-140.18436    -95.62632     -3.5865545  -59.446598 ]
 [-119.264595   -92.42897     -4.27982    -86.548355 ]
 [ -65.43387     -3.5297403 -108.33574   -157.64508  ]
 [ -65.415955    -3.8230033 -111.58875   -161.621    ]
 [-180.8993    -161.67134    -77.23272     -0.7322664]
 [  -2.8020918  -88.394     -161.12628   -210.3276   ]
 [  -4.171529   -83.65649    -98.34264   -135.52994  ]
 [-154.31055   -103.04908     -7.6279254  -41.967087 ]
 [-209.51389   -179.3525     -93.75351     -4.171521 ]
 [-135.17406    -90.617714    -0.8169305  -77.32392  ]
 [-141.30368   -106.51258    -15.940049   -23.444386 ]

In [36]:
smax = tf.nn.softmax(logits, )
preds = tf.math.argmax(smax, axis=1)
probs = tf.reduce_max(smax, 1) 

In [40]:
print('preds:', preds)
print('preds:', probs)

preds: tf.Tensor([1 1 1 1 3 3 1 2 2 1 1 3 0 0 2 3 2 2 3 1 0 0 3 0 1 1 1 1 0 1 0 2], shape=(32,), dtype=int64)
preds: tf.Tensor(
[1.         1.         1.         1.         0.9563847  1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.99944955
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.        ], shape=(32,), dtype=float32)


In [46]:
print('oset.centroids:',oset.centroids)

oset.centroids: tf.Tensor(
[[0.0000000e+00 4.2018835e-02 0.0000000e+00 7.5073738e+00 2.9848788e+00
  0.0000000e+00 5.9337478e+00 3.0970883e+00 4.7078938e+00 5.1352601e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 1.4536023e+01 3.4672394e+00
  6.4453125e+00]
 [0.0000000e+00 6.2599664e+00 0.0000000e+00 1.1224109e+01 2.5894647e+00
  0.0000000e+00 6.1635613e+00 2.2290489e-01 5.6645598e+00 3.8178229e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 1.5476877e+01 6.0169353e+00
  3.0685852e+00]
 [0.0000000e+00 4.1064219e+00 0.0000000e+00 1.0502010e+01 2.1169436e+00
  0.0000000e+00 6.6549835e+00 1.9764199e-03 3.4076446e-01 6.0818281e-02
  0.0000000e+00 0.0000000e+00 0.0000000e+00 1.7405222e+01 8.4182948e-02
  9.9504930e-01]
 [0.0000000e+00 1.7120099e+00 0.0000000e+00 1.5485305e+01 6.2190514e-02
  0.0000000e+00 2.0054388e+00 3.6714032e-02 1.1385172e-01 1.2439347e-01
  0.0000000e+00 0.0000000e+00 0.0000000e+00 1.7535822e+01 1.5839231e-01
  4.3307877e+00]], shape=(4, 16), dtype=float32)


In [ ]:
# centroids are having only 4 rows , whereas labels are rows equivallent to batch
        # pick-up the centroid for each class 
        # label_index from the data set will have all the classes, 32 for a batch
        # for each class cetroid[class_index] will give the centroid of the calss
        # it is basically : [centroids[class_idx] for class_idx in label_indexes]
#         c = centroids[label_indexs]
#         c = tf.gather(centroids, indices=label_indexs)

In [55]:
c = tf.gather(oset.centroids, indices=preds)
# print(c)

In [54]:
euc_dis = tf.norm(features_batch - c, ord='euclidean', axis=1) 
print(euc_dis)

tf.Tensor(
[1.8777233 1.456784  4.964005  1.7292683 6.891102  2.5988834 1.2315795
 1.89382   2.0687726 1.8787605 1.9552501 0.8557257 1.673945  2.042432
 2.76187   2.0424302 0.903842  3.992499  1.7192796 1.8713579 1.7164923
 2.042432  1.9471519 1.7143115 4.964005  1.5410141 1.3176279 1.8787833
 2.042432  4.964005  2.0654626 2.2751262], shape=(32,), dtype=float32)


In [57]:
# d = tf.gather(oset.delta_points, indices=preds)

In [58]:
euc_dis >=oset.delta_points

<tf.Tensor: shape=(1, 4, 32), dtype=bool, numpy=
array([[[ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  

---------------------------------------------------------------------------
NotImplementedError                       Traceback (most recent call last)
C:\Users\BHUJAY~1\AppData\Local\Temp/ipykernel_24404/2131960519.py in <module>
----> 1 oset.train(train_data)

C:\Users\BHUJAY~1\AppData\Local\Temp/ipykernel_24404/3835535165.py in train(self, data_train, epochs)
     35                 logseq_batch, label_batch = batch
     36                 ## (32, 32, 64), (32, 4)
---> 37                 batch_loss, t_loss = self.train_step(criterion_boundary, 
     38                                                      logseq_batch, label_batch)
     39                 tr_loss += t_loss

~\anaconda3\envs\env3\lib\site-packages\tensorflow\python\eager\def_function.py in __call__(self, *args, **kwds)
    778       else:
    779         compiler = "nonXla"
--> 780         result = self._call(*args, **kwds)
    781 
    782       new_tracing_count = self._get_tracing_count()

~\anaconda3\envs\env3\lib\site-packages\tensorflow\python\eager\def_function.py in _call(self, *args, **kwds)
    821       # This is the first call of __call__, so we have to initialize.
    822       initializers = []
--> 823       self._initialize(args, kwds, add_initializers_to=initializers)
    824     finally:
    825       # At this point we know that the initialization is complete (or less

~\anaconda3\envs\env3\lib\site-packages\tensorflow\python\eager\def_function.py in _initialize(self, args, kwds, add_initializers_to)
    694     self._graph_deleter = FunctionDeleter(self._lifted_initializer_graph)
    695     self._concrete_stateful_fn = (
--> 696         self._stateful_fn._get_concrete_function_internal_garbage_collected(  # pylint: disable=protected-access
    697             *args, **kwds))
    698 

~\anaconda3\envs\env3\lib\site-packages\tensorflow\python\eager\function.py in _get_concrete_function_internal_garbage_collected(self, *args, **kwargs)
   2853       args, kwargs = None, None
   2854     with self._lock:
-> 2855       graph_function, _, _ = self._maybe_define_function(args, kwargs)
   2856     return graph_function
   2857 

~\anaconda3\envs\env3\lib\site-packages\tensorflow\python\eager\function.py in _maybe_define_function(self, args, kwargs)
   3211 
   3212       self._function_cache.missed.add(call_context_key)
-> 3213       graph_function = self._create_graph_function(args, kwargs)
   3214       self._function_cache.primary[cache_key] = graph_function
   3215       return graph_function, args, kwargs

~\anaconda3\envs\env3\lib\site-packages\tensorflow\python\eager\function.py in _create_graph_function(self, args, kwargs, override_flat_arg_shapes)
   3063     arg_names = base_arg_names + missing_arg_names
   3064     graph_function = ConcreteFunction(
-> 3065         func_graph_module.func_graph_from_py_func(
   3066             self._name,
   3067             self._python_function,

~\anaconda3\envs\env3\lib\site-packages\tensorflow\python\framework\func_graph.py in func_graph_from_py_func(name, python_func, args, kwargs, signature, func_graph, autograph, autograph_options, add_control_dependencies, arg_names, op_return_value, collections, capture_by_value, override_flat_arg_shapes)
    984         _, original_func = tf_decorator.unwrap(python_func)
    985 
--> 986       func_outputs = python_func(*func_args, **func_kwargs)
    987 
    988       # invariant: `func_outputs` contains only Tensors, CompositeTensors,

~\anaconda3\envs\env3\lib\site-packages\tensorflow\python\eager\def_function.py in wrapped_fn(*args, **kwds)
    598         # __wrapped__ allows AutoGraph to swap in a converted function. We give
    599         # the function a weak reference to itself to avoid a reference cycle.
--> 600         return weak_wrapped_fn().__wrapped__(*args, **kwds)
    601     weak_wrapped_fn = weakref.ref(wrapped_fn)
    602 

~\anaconda3\envs\env3\lib\site-packages\tensorflow\python\eager\function.py in bound_method_wrapper(*args, **kwargs)
   3733     # However, the replacer is still responsible for attaching self properly.
   3734     # TODO(mdan): Is it possible to do it here instead?
-> 3735     return wrapped_fn(*args, **kwargs)
   3736   weak_bound_method_wrapper = weakref.ref(bound_method_wrapper)
   3737 

~\anaconda3\envs\env3\lib\site-packages\tensorflow\python\framework\func_graph.py in wrapper(*args, **kwargs)
    971           except Exception as e:  # pylint:disable=broad-except
    972             if hasattr(e, "ag_error_metadata"):
--> 973               raise e.ag_error_metadata.to_exception(e)
    974             else:
    975               raise

NotImplementedError: in user code:

    C:\Users\BHUJAY~1\AppData\Local\Temp/ipykernel_24404/3835535165.py:50 train_step  *
        loss, self.delta = criterion_boundary(features_batch,
    C:\Users\Bhujay_ROG\anaconda3\envs\env3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:985 __call__  **
        outputs = call_fn(inputs, *args, **kwargs)
    C:\Users\Bhujay_ROG\MyDev\OCLog\oclog\boundary_loss.py:34 call  **
        logits =  euclidean_metric(features, centroids)
    C:\Users\Bhujay_ROG\MyDev\OCLog\oclog\boundary_loss.py:12 euclidean_metric
        a = np.expand_dims(a, 1)
    <__array_function__ internals>:5 expand_dims
        
    C:\Users\Bhujay_ROG\anaconda3\envs\env3\lib\site-packages\numpy\lib\shape_base.py:591 expand_dims
        a = asanyarray(a)
    C:\Users\Bhujay_ROG\anaconda3\envs\env3\lib\site-packages\numpy\core\_asarray.py:136 asanyarray
        return array(a, dtype, copy=False, order=order, subok=True)
    C:\Users\Bhujay_ROG\anaconda3\envs\env3\lib\site-packages\tensorflow\python\framework\ops.py:845 __array__
        raise NotImplementedError(

    NotImplementedError: Cannot convert a symbolic Tensor (log_classifier/log_seq_encoder/dense/Relu:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
var1 = tf.Variable(10.0)
loss = lambda: (var1 ** 2)/2.0       # d(loss)/d(var1) == var1
step_count = opt.minimize(loss, [var1]).numpy()
# The first step is `-learning_rate*sign(grad)`
var1.numpy()
